In [1]:
import json
import re
import string
import numpy as np
import pandas as pd
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory # Rumus Library
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import string # Rumus Library
import sys # Rumus supaya baca command promp


stemmer = StemmerFactory().create_stemmer() # Object stemmer
remover = StopWordRemoverFactory().create_stop_word_remover() # objek stopword

In [2]:
data = pd.read_excel('dok.xlsx');
list_dokumen = data.Bahan.tolist()
list_dokumen = [tem.replace(u"\xa0",u"") for tem in list_dokumen]
list_dokumen = [str(tem) for tem in list_dokumen]
list_dokumen

['50 gram Beras 25 gram Brokoli 1/2 sendok teh Garam 400 cc Kaldu Ayam 2 sendok teh Minyak 50 gram Tempe 15 gram Teri Nasi Tawar',
 '5 buah Bakso 1 batang Daun Bawang  1 siung Bawang Putih 1 sendok teh Garam 500 cc Kaldu 1 sendok makan Minyak Sayur 1 buah Oyong 1 batang Seledri 50 gram Soun 1 buah Tahu',
 '1 buah Jagung Manis 100 gram Kacang Polong 50 gram Keju Cheddar 125 gram Margarin 1/2 sendok teh Merica Bubuk 200 ml Susu Cair 2 butir Telur 350 gram Tepung Terigu 100 gram Wortel',
 '1 batang Daun Bawang 1 sendok teh Garam Halus 500 gram Kentang 100 gram Kornet Kalengan 1 sendok makan Margarin',
 '1 siung Bawang Putih 1 sendok teh Garam 1 buah Hati Ayam Rebus 50 gram Jagung Manis 200 ml Kaldu Ayam 1 sendok teh Mentega 50 ml Susu 1 sendok teh Tepung Terigu 20 gram Wortel',
 '1 sendok teh Bawang Bombay 50 gram Brokoli 50 gram Dada Ayam 200 ml Kaldu Ayam 50 gram Kentang 1 sendok teh Minyak Zaitun 50 gram Tahu Sutra',
 '25 gram Daging Ayam 25 gram Kacang Polong 300 cc Kaldu 10 gram Keju

In [11]:
def preprocess(doc): #fungsi untuk preprocesing dokumen
    listStem = [] # variabel untuk menampung data preprocesing
    for i in doc:
        lowerku = i.lower() # membuat huruf kecil
        s = re.sub(r'[^\w\s]','',lowerku) # normalisasi text dari code
        textStemmed = stemmer.stem(s) # steming kata
        textClean = remover.remove(textStemmed) # membuang kata tdak penting
        if textClean != "-": 
            listStem.append(textClean) #menambah data kata yang sudah di steming di dalam variabel liststem
    return listStem # mengembalikan nilai listStem

def replaceMultiple(mainString, toBeReplaces, newString):
    # Iterate over the strings to be replaced
    for elem in toBeReplaces :
        # Check if string is in the main string
        if elem in mainString :
            # Replace the string
            mainString = mainString.replace(elem, newString)
    
    return  mainString

In [12]:
pre_bahan = preprocess(list_dokumen)
pre_bahan

['50 gram beras 25 gram brokoli 12 sendok teh garam 400 cc kaldu ayam 2 sendok teh minyak 50 gram tempe 15 gram teri nasi tawar',
 '5 buah bakso 1 batang daun bawang 1 siung bawang putih 1 sendok teh garam 500 cc kaldu 1 sendok makan minyak sayur 1 buah oyong 1 batang seledri 50 gram soun 1 buah tahu',
 '1 buah jagung manis 100 gram kacang polong 50 gram keju cheddar 125 gram margarin 12 sendok teh merica bubuk 200 ml susu cair 2 butir telur 350 gram tepung terigu 100 gram wortel',
 '1 batang daun bawang 1 sendok teh garam halus 500 gram kentang 100 gram kornet kaleng 1 sendok makan margarin',
 '1 siung bawang putih 1 sendok teh garam 1 buah hati ayam rebus 50 gram jagung manis 200 ml kaldu ayam 1 sendok teh mentega 50 ml susu 1 sendok teh tepung terigu 20 gram wortel',
 '1 sendok teh bawang bombay 50 gram brokoli 50 gram dada ayam 200 ml kaldu ayam 50 gram kentang 1 sendok teh minyak zaitun 50 gram tahu sutra',
 '25 gram daging ayam 25 gram kacang polong 300 cc kaldu 10 gram keju paru

In [14]:
list_bahan = []
for sampel in pre_bahan:
    result = ''.join([i for i in sampel if not i.isdigit()])
    otherStr = replaceMultiple(result, ['gram','ml','cc','buah','sendok teh','sendok makan','sendok takar','butir','cangkir','siung','batang'] ,"")
    a = re.sub(' +',' ',otherStr) #untuk menghilangkan double spasi dikalimat
    b = a.lstrip() #untuk hilangkan spasi di depan kalimat
    list_bahan.append(b)
print(list_bahan)

['beras brokoli garam kaldu ayam minyak tempe teri nasi tawar', 'bakso daun bawang bawang putih garam kaldu minyak sayur oyong seledri soun tahu', 'jagung manis kacang polong keju cheddar margarin merica bubuk susu cair telur tepung terigu wortel', 'daun bawang garam halus kentang kornet kaleng margarin', 'bawang putih garam hati ayam rebus jagung manis kaldu ayam mentega susu tepung terigu wortel', 'bawang bombay brokoli dada ayam kaldu ayam kentang minyak zaitun tahu sutra', 'daging ayam kacang polong kaldu keju parut makaroni tahu potong tomat', 'garam kentang margarin merica bubuk pala bubuk susu cair', 'beras daging ayam kaldu ayam keju parut santan tahu tomat wortel', 'bayam butter tawar daun bawang kacang polong beku kaldu ubi']


In [15]:
import math

def dice_distance(a, b):
    c = a.intersection(b)
    return float(2 * len(c)) / (len(a) + len(b))

def search(query, datanya):
    lol = preprocess([query])
    alldata = []
    alldata.append(set(lol[0].split(" ")))
    for i in datanya:
        lol2 = preprocess([i])
        alldata.append(lol2[0].split(" "))
    
    score = []
#     print alldata
    for tf in range(1,len(alldata)):
        score.append(dice_distance(alldata[0],alldata[tf]))
#     print len(alldata)
#         print score
    return score

In [16]:
semuanya = []
n = search("daging ayam makaroni tahu", list_bahan)
frame = pd.DataFrame(data['Bahan'])
# print data['Temuan']
frame['Score'] = n
frame

,Bahan,Score
0,50 gram Beras 25 gram Brokoli 1/2 sendok teh G...,0.142857
1,5 buah Bakso 1 batang Daun Bawang 1 siung Baw...,0.117647
2,1 buah Jagung Manis 100 gram Kacang Polong 50 ...,0.000000
3,1 batang Daun Bawang 1 sendok teh Garam Halus ...,0.000000
4,1 siung Bawang Putih 1 sendok teh Garam 1 buah...,0.105263
5,1 sendok teh Bawang Bombay 50 gram Brokoli 50 ...,0.250000
6,25 gram Daging Ayam 25 gram Kacang Polong 300 ...,0.533333
7,1/2 sendok teh Garam 750 gram Kentang 3 sendok...,0.000000
8,2 sendok makan Beras 25 gram Daging Ayam 300 m...,0.400000
9,30 gram Bayam 25 gram Butter Tawar 30 gram Dau...,0.000000


In [17]:
i = 0
alldrop = []
for testscore in frame['Score']:
            if testscore<=0.5:
                alldrop.append(i)
#                 frame.drop(frame.index[i])
            i = i + 1

In [25]:
newframe = frame.drop(frame.index[alldrop])
rank = newframe.sort_values(by=['Score'], ascending=[False])
newframe
rank

,Bahan,Score,Keterangan
6,25 gram Daging Ayam 25 gram Kacang Polong 300 ...,0.533333,Relevan


In [32]:
ket = []
for data in frame['Score']:
    if data >= 0.5:
        z = "Relevan"
    else:
        z = "Tidak Relevan"
    ket.append(z) # Harus di simpen dulu di list biar jumlahnya sama seperti data frame
frame['Keterangan'] = ket # baru di assignment ket ke dalam frame['keterangan']

In [33]:
rank = frame.sort_values(by=['Score'], ascending=[False])
rank

,Bahan,Score,Keterangan
6,25 gram Daging Ayam 25 gram Kacang Polong 300 ...,0.533333,Relevan
8,2 sendok makan Beras 25 gram Daging Ayam 300 m...,0.400000,Tidak Relevan
5,1 sendok teh Bawang Bombay 50 gram Brokoli 50 ...,0.250000,Tidak Relevan
0,50 gram Beras 25 gram Brokoli 1/2 sendok teh G...,0.142857,Tidak Relevan
1,5 buah Bakso 1 batang Daun Bawang 1 siung Baw...,0.117647,Tidak Relevan
4,1 siung Bawang Putih 1 sendok teh Garam 1 buah...,0.105263,Tidak Relevan
2,1 buah Jagung Manis 100 gram Kacang Polong 50 ...,0.000000,Tidak Relevan
3,1 batang Daun Bawang 1 sendok teh Garam Halus ...,0.000000,Tidak Relevan
7,1/2 sendok teh Garam 750 gram Kentang 3 sendok...,0.000000,Tidak Relevan
9,30 gram Bayam 25 gram Butter Tawar 30 gram Dau...,0.000000,Tidak Relevan
